In [15]:
import gym
from matplotlib import pyplot as plt
import numpy as np # used for arrays
import time # to get the time
import math # needed for calculations

In [16]:
# env = gym.make('MountainCar-v0', render_mode="human")
# # reset environment and get initial observation/state
# # Observation/state is a tuple of (position, velocity)
# obs = env.reset()
# print("initial observation:", obs)

# # possible 3 actions
# # {0: "accelerate to left", "1": "do nothing", "2": "accelerate to right"}
# print("possible actions:", env.action_space.n)


# # reinforcement learning is all
# # about learing to take good actions
# # from a given state/observation
# # right now taking a random action
# def policy(observation):
#     return env.action_space.sample()


# # take 5 random actions/steps
# for _ in range(5):

#     done = False

#     # to render environment for visual inspection
#     # when you train, you can skip rendering to speed up
#     while not done: 
#         env.render()

#         # based on curret policy, use the current observation
#         # to find the best action to take.
#         action = policy(obs)
#         print("\ntaking action:", action)

#         # pass the action to env which will return back
#         # with new state/"observation" and "reward"
#         # there is a "done" flag which is true when game ends
#         # "info" provides some diagnostic information
#         obs, reward, done, info, _ = env.step(action)
#         print("got reward: {0}. New state/observation is: {1}".format(reward, obs))

# # close the enviroment
# env.close()

In [17]:
LEARNING_RATE = 0.1

DISCOUNT = 0.95
EPISODES = 100
total = 0
total_reward = 0
prior_reward = 0

Observation = [50, 50, 50, 50]
np_array_win_size = np.array([0.12, 0.4, 0.012, 0.4])

epsilon = 1

epsilon_decay_value = 0.99995

In [18]:
# env = gym.make('CartPole-v1', render_mode="human")
# obs = env.reset()
# print("initial observation:", obs)
# print("possible actions:", env.action_space.n)


# def policy(observation):
#     return env.action_space.sample()


# for i in range(5):
#     env.reset()
#     done = False

#     while not done:
#         env.render()

#         action = policy(obs)
#         print("\ntaking action:", action)

#         obs, reward, done, _, info = env.step(action)
#         print(done)
#         print("got reward: {0}. New state/observation is: {1}".format(reward, obs))


# # close the enviroment
# env.close()

In [19]:
env = gym.make("CartPole-v1", render_mode="human")
print(env.action_space.n)

2


In [20]:
q_table = np.random.uniform(low=0, high=1, size=(Observation + [env.action_space.n]))
q_table.shape

(50, 50, 50, 50, 2)

In [21]:
def get_discrete_state(state):
    discrete_state = np.divide(state,np_array_win_size) + 25
    print(state, " -> ", discrete_state.astype(int))
    return tuple(discrete_state.astype(int))

In [22]:
for episode in range(EPISODES + 1): #go through the episodes
    t0 = time.time() #set the initial time
    discrete_state = get_discrete_state(env.reset()[0]) #get the discrete start for the restarted environment 
    done = False
    episode_reward = 0 #reward starts as 0 for each episode

    if episode % 2000 == 0: 
        print("Episode: " + str(episode))

    while not done: 
        print("actions: ", q_table[discrete_state])
        if np.random.random() > epsilon:

            action = np.argmax(q_table[discrete_state]) #take action with highest known reward
        else:

            action = np.random.randint(0, env.action_space.n) #do a random ation

        new_state, reward, done, _, _ = env.step(action) #step action to get new states, reward, and the "done" status.

        episode_reward += reward #add the reward

        new_discrete_state = get_discrete_state(new_state)

        if episode % 10 == 0: #render
            env.render()

        if (not done) and new_discrete_state in q_table: #update q-table
            # make sure this is within range
            max_future_q = np.max(q_table[new_discrete_state])

            current_q = q_table[discrete_state + (action,)]

            new_q = (1 - LEARNING_RATE) * current_q + LEARNING_RATE * (reward + DISCOUNT * max_future_q)

            q_table[discrete_state + (action,)] = new_q

        discrete_state = new_discrete_state

    if epsilon > 0.05: #epsilon modification
        if episode_reward > prior_reward and episode > 10000:
            epsilon = math.pow(epsilon_decay_value, episode - 10000)

            if episode % 500 == 0:
                print("Epsilon: " + str(epsilon))

    t1 = time.time() #episode has finished
    episode_total = t1 - t0 #episode total time
    total = total + episode_total

    total_reward += episode_reward #episode total reward
    prior_reward = episode_reward

    if episode % 1000 == 0: #every 1000 episodes print the average time and the average reward
        mean = total / 1000
        print("Time Average: " + str(mean))
        total = 0

        mean_reward = total_reward / 1000
        print("Mean Reward: " + str(mean_reward))
        total_reward = 0


env.close()

[-0.02191418 -0.0456269   0.00144235  0.0420091 ]  ->  [24 24 25 25]
Episode: 0
actions:  [0.58915896 0.72185271]
[-0.02282672  0.14947434  0.00228253 -0.2502184 ]  ->  [24 25 25 24]
actions:  [0.63501805 0.44904095]
[-0.01983723  0.3445636  -0.00272183 -0.5421805 ]  ->  [24 25 24 23]
actions:  [0.85467825 0.92985772]
[-0.01294596  0.5397237  -0.01356544 -0.83571976]  ->  [24 26 23 22]


/opt/anaconda3/envs/apress/lib/python3.8/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
/var/folders/dn/9sjhn7m57nz120m7skmp4_7r0000gn/T/ipykernel_49611/1049958202.py:28: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  if (not done) and new_discrete_state in q_table: #update q-table


actions:  [0.32385563 0.48669482]
[-0.00215148  0.34478968 -0.03027984 -0.5473337 ]  ->  [24 25 22 23]
actions:  [0.94309996 0.0649532 ]
[ 0.00474431  0.5403237  -0.04122651 -0.8494012 ]  ->  [25 26 21 22]
actions:  [0.36391743 0.24175221]
[ 0.01555078  0.73598284 -0.05821454 -1.1547577 ]  ->  [25 26 20 22]
actions:  [0.35604682 0.6479372 ]
[ 0.03027044  0.9318136  -0.08130969 -1.4651117 ]  ->  [25 27 18 21]
actions:  [0.9303758  0.38707015]
[ 0.04890671  1.1278318  -0.11061193 -1.7820463 ]  ->  [25 27 15 20]
actions:  [0.5680944  0.12000714]
[ 0.07146335  0.934114   -0.14625286 -1.525699  ]  ->  [25 27 12 21]
actions:  [0.97799861 0.43716366]
[ 0.09014563  1.1306672  -0.17676683 -1.8602265 ]  ->  [25 27 10 20]
actions:  [0.84620788 0.03497485]
[ 0.11275897  1.3272334  -0.21397136 -2.2021804 ]  ->  [25 28  7 19]
Time Average: 0.0006142406463623047
Mean Reward: 0.011
[0.03099729 0.02277919 0.02288959 0.01098829]  ->  [25 25 26 25]
actions:  [0.75514969 0.60630728]
[ 0.03145288 -0.172663